## Importation des bibliothéques

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import keras
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD



from IPython.core.display import HTML 

## Importation de la data

In [ ]:
df = pd.read_csv('../input/voicegender/voice.csv')

## Preprocessing de la data

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
display(HTML(df.head(100).to_html()))

In [ ]:
df.label.value_counts()

In [ ]:
check_for_nan = df.isnull().values.any()
print (check_for_nan)

In [ ]:
df.info()

### Visualisation & Corrélation

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(13,13))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(df.iloc[:,:-1].astype(float).corr(),vmax=1.0, square=True,cmap="RdBu_r",annot=True)

D'après le tableau de corrélation, j'ai constaté qu'il y'a une parfaite corrélation entre 'maxdom' et 'dfrange', aussi entre 'meanfreq' et 'centroid'. Apprement encore, j'ai constaté s’il y'a une forte corrélation entre 'skew' et 'kurt'. Pour plus d'expliciter je vais vous présenter une visualisation illustrant la dépendance entre ces variables.

In [ ]:
sns.relplot(x='maxdom',y='dfrange',hue='label',kind='scatter',data=df)

In [ ]:
sns.relplot(x='meanfreq',y='centroid',hue='label',kind='scatter',data=df)

In [ ]:
sns.relplot(x='skew',y='kurt',hue='label',kind='scatter',data=df)

Alors je vais supprimer 'maxdom','centroid', 'kurt'.

In [ ]:
df=df.drop(['maxdom','centroid','kurt'],axis=1)

Fractionnement

Repartition des variable (entrées / sortie)

In [ ]:
X=df.drop(['label'],axis=1)
y=df['label']

### Encoding

Convertir la variable categorical en variable numérique (binaire).

In [ ]:
y=y.map({'male':0,'female':1})

### Standardisation

Mettre à la même échelle les variables pour ne pas biaisé le système des données et fiabiliser les calculs et résultats.

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
fig = sns.FacetGrid(df, hue="label", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "meanfun", shade=True)
fig.add_legend()

In [ ]:
fig = sns.FacetGrid(df, hue="label", aspect=3, palette="Set2") # aspect=3 permet d'allonger le graphique
fig.map(sns.kdeplot, "meanfreq", shade=True)
fig.add_legend()

J'ai fait ces illustrations dessus pour vous montrer que certains dont certaines variables, il est accessible de classifier entre les voix males et females alors que dans des autres variables, on constate une grande interférence, alors pour cela, je vais exploiter plusieurs modèles afin d'en trouver le mieux qui nous permettra d'éliminer ces interférences.

Mais tous d'abord, je vais faire un splitting pour repartir nos données d'entrainement et ceux de test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

### Machine learning

#### Tree

In [ ]:
from sklearn import tree
dtc=tree.DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_dtc=dtc.predict(X_test)
print(accuracy_score(y_test,y_dtc))
print(confusion_matrix(y_test,y_dtc))
print(classification_report(y_test,y_dtc))

#### Ensemble

In [ ]:
from sklearn import ensemble
rf=ensemble.RandomForestClassifier()
rf.fit(X_train,y_train)
y_rf=rf.predict(X_test)
print(accuracy_score(y_test,y_rf))
print(confusion_matrix(y_test,y_rf))
print(classification_report(y_test,y_rf))

In [ ]:
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, y_train)
y_svm = svm.predict(X_test)
print(accuracy_score(y_test, y_svm))
print(confusion_matrix(y_test, y_svm))
print(classification_report(y_test, y_svm))

#### deep learning _ Reseau de neuron

In [ ]:
n_cols = X_train.shape[1]
y_cat = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=1)

In [ ]:
hist = History()

model = Sequential()

model.add(Dense(1000, activation='relu', input_dim = n_cols))
model.add(Dense(700, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(400, activation='relu'))


model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train=model.fit(X_train, y_train, epochs = 50, validation_split = .1, callbacks = [hist])

J'ai utilisé une architecture en couches (modèle Sequential), avec des couches denses : j'ai définit un modèle à quatre couches cachées de 1000,700,400,400,400 d'activation relu. les donnees de nombre de neurons c'est d'apres les tests. j'ai choisi qui donne le resultat le plus performant.
J'ai choisi de mettre l'activation de la dernière couche sigmoid et non pas softmax parce qu'on a que 2 classe.
(on interprète la sortie des derniers neurones comme une probabilité d'appartenance au 2 classe.

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
print(train.history['accuracy'])

In [ ]:
print(train.history['val_accuracy']) 

In [ ]:
def plot_scores(train):
    accuracy=train.history['accuracy']
    val_accuracy=train.history['val_accuracy']
    epochs=range(len(accuracy))
    plt.plot(epochs,accuracy,'b',label='Score apprentissage')
    plt.plot(epochs,val_accuracy,'r',label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

In [ ]:
from matplotlib.ticker import FuncFormatter

x = np.arange(4)
accuracy = [94.3,98.1,98.5,98.2]

fig, ax = plt.subplots()
plt.bar(x, accuracy)
plt.xticks(x, ('Tree','Ensemble','SVM','RN'))
ax.set_ylabel('accuracy %')
ax.set_title('Comparaison Modeles')
plt.show()

Franchement les modèles sont puissants et donnent des résultats supers, il y'a des différences indifférentes surtout au niveau du modèle Ensemble, SVM et RN. Personnellement pour ce cas je privilégie SVM.